In [163]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lex-fridman-podcast-transcript/podcastdata_dataset.csv


Importamos el CSV para empezar a trabajar

In [164]:
df = pd.read_csv('/kaggle/input/lex-fridman-podcast-transcript/podcastdata_dataset.csv')

In [179]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

Crear una lista para almacenar las filas del DataFrame final

In [180]:
# Crear una lista para almacenar las filas del DataFrame final
rows = []

In [181]:
# Procesar cada fila del DataFrame
for _, row in df.iterrows():
    text_id = row['id']
    text = row['text']
    
    # Tokenizar el texto en oraciones
    sentences = [sentence.strip() for sentence in sent_tokenize(text) if sentence.strip()]
    
    # Agregar las oraciones al listado con el ID del texto
    for sentence in sentences:
        rows.append({'id': text_id, 'oracion': sentence})


In [183]:
# Crear el nuevo DataFrame
df_sentences = pd.DataFrame(rows)
df_sentences['tokens'] = df_sentences['oracion'].apply(word_tokenize)
df_sentences.tail()

,id,oracion,tokens
443537,325,Is it in the cells?,"[Is, it, in, the, cells, ?]"
443538,325,"There are many, many layers to this as always ...","[There, are, many, ,, many, layers, to, this, ..."
443539,325,So there are chemical networks.,"[So, there, are, chemical, networks, .]"
443540,325,"So for example, gene regulatory networks, right?","[So, for, example, ,, gene, regulatory, networ..."
443541,325,"Which, or basically any kind of chemical pathway","[Which, ,, or, basically, any, kind, of, chemi..."


In [186]:
from gensim.models import Word2Vec

# Crear un modelo Word2Vec
model = Word2Vec(sentences=df_sentences['tokens'], vector_size=100, window=5, min_count=1, workers=4)

In [190]:
import numpy as np

# Función para obtener el embedding promedio de una oración
def get_sentence_embedding(tokens, model):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# Calcular los embeddings para todas las oraciones
df_sentences['embedding'] = df_sentences['tokens'].apply(lambda x: get_sentence_embedding(x, model).tolist())
df_sentences.head()


,id,oracion,tokens,embedding
0,1,"As part of MIT course 6S099, Artificial Genera...","[As, part, of, MIT, course, 6S099, ,, Artifici...","[0.17821069061756134, -0.23301361501216888, -0..."
1,1,He is a professor here at MIT.,"[He, is, a, professor, here, at, MIT, .]","[0.24171370267868042, 0.2438661754131317, -0.4..."
2,1,"He's a physicist, spent a large part of his ca...","[He, 's, a, physicist, ,, spent, a, large, par...","[-0.1599445641040802, 0.7493974566459656, -0.2..."
3,1,But he's also studied and delved into the bene...,"[But, he, 's, also, studied, and, delved, into...","[0.3598919212818146, 0.46108460426330566, -0.3..."
4,1,"Amongst many other things, he is the cofounder...","[Amongst, many, other, things, ,, he, is, the,...","[0.3858562409877777, 0.6571797132492065, -0.42..."


In [192]:
from sklearn.cluster import KMeans
import numpy as np

# Cargar los embeddings
embeddings = np.array(df_sentences['embedding'].tolist())

# Aplicar KMeans para clusterización
num_clusters = 5  # Define el número de clusters según el caso
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_sentences['cluster'] = kmeans.fit_predict(embeddings)
df_sentences.head()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,id,oracion,tokens,embedding,cluster
0,1,"As part of MIT course 6S099, Artificial Genera...","[As, part, of, MIT, course, 6S099, ,, Artifici...","[0.17821069061756134, -0.23301361501216888, -0...",0
1,1,He is a professor here at MIT.,"[He, is, a, professor, here, at, MIT, .]","[0.24171370267868042, 0.2438661754131317, -0.4...",0
2,1,"He's a physicist, spent a large part of his ca...","[He, 's, a, physicist, ,, spent, a, large, par...","[-0.1599445641040802, 0.7493974566459656, -0.2...",0
3,1,But he's also studied and delved into the bene...,"[But, he, 's, also, studied, and, delved, into...","[0.3598919212818146, 0.46108460426330566, -0.3...",0
4,1,"Amongst many other things, he is the cofounder...","[Amongst, many, other, things, ,, he, is, the,...","[0.3858562409877777, 0.6571797132492065, -0.42...",0


In [196]:
from sklearn.metrics.pairwise import cosine_similarity

# Consulta de ejemplo
query = "artificial intelligence"
query_tokens = word_tokenize(query.lower())  # Tokenizar la consulta

# Calcular el embedding de la consulta
query_embedding = get_sentence_embedding(query_tokens, model).reshape(1, -1)

# Calcular similitudes entre la consulta y los embeddings de las oraciones
df_sentences['similarity'] = df_sentences['embedding'].apply(lambda x: cosine_similarity(query_embedding, np.array(x).reshape(1, -1))[0][0])

# Ordenar las oraciones por similitud (de mayor a menor)
resultados = df.sort_values(by='similarity', ascending=False)

# Mostrar las oraciones más relevantes
print(resultados[['id', 'oracion', 'similarity']].head(10))


KeyError: 'similarity'